In [2]:
import numpy as np
import pandas as pd

### processing 1 start

In [ ]:
df=pd.read_csv('./dataset_TSMC2014_NYC.csv',parse_dates=['utcTimestamp'])
df.head()

In [ ]:
import datetime

In [ ]:
df['localtime']=0
for index in range(len(df)):
    dt=datetime.timedelta(minutes=int(df.timezoneOffset[index]))
    df['localtime'][index]=df.utcTimestamp[index]+dt
df.head()

In [ ]:
df['day']=df.localtime.apply(lambda x: x.dayofweek)
df.head()

In [ ]:
# len(df)/len(set(df.userId.to_numpy()))  #平均每个用户长度为210
# print([len(df[df.userId==x]) for x in list(set(df.userId.to_numpy()))])
# min([len(df[df.userId==x]) for x in list(set(df.userId.to_numpy()))])

### 去掉序列长度小于100的用户数据

In [ ]:
userId=df.userId.to_numpy()
u_categories=list(set(userId))

uids_filter=[]    
for uid in u_categories:
    if len(df[df.userId==uid])<=100:
        uids_filter.append(uid)
for uid in uids_filter:
    index=df[df.userId==uid].index
    df.drop(index=index,inplace=True)

In [ ]:
df.sort_values(by=['userId','localtime'],ignore_index=True,inplace=True)    #重新给index排序
df

In [ ]:
# #检查是否去掉了一些用户
# userId=df.userId.to_numpy()
# u_categories=list(set(userId))
# len(u_categories)

### split everyday to 8 intervals

In [ ]:
def Timelabel(x):
    x=x.hour*60+x.minute+x.second/60
    # hour=datetime.time.fromisoformat(x).hour
    intervals=np.linspace(180,24*60,8)  #interval is 180 minutes,so timelabel from 0 to 7.

    return np.digitize(x,intervals)  

df['daytimelabel']=df.localtime.apply(Timelabel)
df.head()

In [ ]:
df.drop(columns=['timezoneOffset','utcTimestamp'],inplace=True)
df.head()

### interval=72 hours 将每个用户的记录序列切分为sessions

In [ ]:
# df[df.userId==1]

In [ ]:
# df.to_csv('datacheck_1.csv',index=False)

### Processing 1 over,2 start

In [55]:
df = pd.read_csv('datacheck_1.csv',parse_dates=['localtime'])

In [ ]:
# x=[[1,2],[3,4,5],[]]
# x[-1].append(df[['venueCategory','day','daytimelabel']].iloc[2].to_list())
# x

In [56]:
data_all={}

In [57]:
uid_categories=list(set(df.userId.to_numpy()))  #encode uid,vid,vcid from 0 to length
vid_categories=list(set(df.venueId.to_numpy()))     
vcid_categories=list(set(df.venueCategoryId.to_numpy()))

df['userId']=df.userId.apply(lambda x:uid_categories.index(x))    #重新编码
df['venueId']=df.venueId.apply(lambda x:vid_categories.index(x))
df['venueCategoryId']=df.venueCategoryId.apply(lambda x:vcid_categories.index(x))

uid_categories=list(set(df.userId.to_numpy()))
vid_categories=list(set(df.venueId.to_numpy()))   
vcid_categories=list(set(df.venueCategoryId.to_numpy()))

data_all['uid_categories']=uid_categories
data_all['vid_categories']=vid_categories
data_all['vcid_categories']=vcid_categories

In [58]:
vcid_vc={}   #dict from vcid to vc
for index in df.index:
    vcid=df.venueCategoryId[index]
    vc=df.venueCategory[index]
    if vcid not in vcid_vc:
        vcid_vc[vcid]=[vc,0]
    else:
        vcid_vc[vcid][1]+=1
data_all['vcid_vc']=vcid_vc

In [7]:
vid_vcid={}
for index in df.index:
    vcid=df.venueCategoryId[index]
    vid=df.venueId[index]
    if vid not in vid_vcid:
        vid_vcid[vid]=vcid
data_all['vid_vcid']=vid_vcid

In [59]:
vid_vc={}
for index in df.index:
    vid=df.venueId[index]
    vc=df.venueCategory[index]
    if vid not in vid_vc:
        vid_vc[vid]=vc
data_all['vid_vc']=vid_vc

In [60]:
gps_seq=df[['latitude','longitude']].to_numpy()

In [61]:
vid_gps={}
for index in df.index:
    gps=gps_seq[index]
    vid=df.venueId[index]
    if vid not in vid_gps:
        vid_gps[vid]=gps
data_all['vid_gps']=vid_gps

In [10]:
# lat=gps_seq[:,0]
# lat_min=min(lat)
# lat_max=max(lat)
# lon=gps_seq[:,1]
# lon_min=min(lon)
# lon_max=max(lon)

In [20]:
# import gmplot
# gmap = gmplot.GoogleMapPlotter(lat[0], lon[0], 11)
# user=1
# lat1=df[df.userId==user].latitude.to_numpy()
# lon1=df[df.userId==user].longitude.to_numpy()
# gmap.scatter(lat1,lon1,color='red',size=40)
# gmap.plot(lat1, lon1)
# gmap.draw("data_map_"+str(user)+".html")

In [22]:
import datetime

In [62]:
session_dt=datetime.timedelta(hours=72)
S_u={uid:[] for uid in uid_categories}
for uid in uid_categories:
    df_sub=df[df.userId==uid]
    df_sub.sort_values(by=['localtime'],ignore_index=True,inplace=True)
    for index in df_sub.index:
        if index is 0:  #每个用户最开始时要添加session
            S_u[uid].append([]) #append a session
            r=df_sub[['userId','venueId','day','daytimelabel']].iloc[index].to_list()
            S_u[uid][-1].append(r)   #add a record into current session
            continue
        if df_sub.localtime[index]-df.localtime[index-1]>session_dt:    #时间间隔超过24小时添加新session
            S_u[uid].append([]) #append a session
            r=df_sub[['userId','venueId','day','daytimelabel']].iloc[index].to_list()
            S_u[uid][-1].append(r)   #add a record into current session
            continue
        if len(S_u[uid][-1])>=11: #session长度超过10则添加新session
            S_u[uid].append([]) #append a session
            r=df_sub[['userId','venueId','day','daytimelabel']].iloc[index].to_list()
            S_u[uid][-1].append(r)   #add a record into current session
            continue

        r=df_sub[['userId','venueId','day','daytimelabel']].iloc[index].to_list() #普通的连续记录直接添加到当前session
        S_u[uid][-1].append(r)   #add a record into current session

In [63]:
data_all['sessions_raw']=S_u

In [65]:
S_u[0]

[[[0, 16228, 2, 6],
  [0, 3354, 5, 4],
  [0, 12527, 6, 4],
  [0, 10493, 6, 5],
  [0, 12337, 0, 4],
  [0, 10662, 0, 6],
  [0, 35153, 0, 7],
  [0, 25905, 1, 4],
  [0, 8051, 3, 4],
  [0, 25905, 4, 3],
  [0, 22734, 4, 7]],
 [[0, 32519, 4, 7],
  [0, 20739, 5, 0],
  [0, 6062, 5, 3],
  [0, 16499, 5, 4],
  [0, 20513, 5, 5],
  [0, 27937, 5, 5],
  [0, 8132, 5, 6],
  [0, 19569, 5, 6],
  [0, 19569, 5, 6],
  [0, 30963, 0, 3],
  [0, 7757, 0, 4]],
 [[0, 9620, 2, 4],
  [0, 14554, 2, 7],
  [0, 8051, 3, 5],
  [0, 28252, 3, 6],
  [0, 11198, 3, 7],
  [0, 29923, 4, 6],
  [0, 21243, 5, 4],
  [0, 18353, 5, 4],
  [0, 16894, 5, 4],
  [0, 31825, 5, 5],
  [0, 9629, 5, 5]],
 [[0, 7785, 5, 5], [0, 27578, 5, 7], [0, 3354, 6, 4], [0, 8051, 0, 4]],
 [[0, 16228, 4, 7], [0, 3354, 6, 5], [0, 22242, 1, 6], [0, 35153, 1, 6]],
 [[0, 37175, 5, 3]],
 [[0, 28964, 1, 4],
  [0, 18990, 4, 4],
  [0, 16286, 4, 7],
  [0, 29831, 5, 4],
  [0, 2834, 5, 6],
  [0, 11273, 5, 6],
  [0, 26798, 6, 6],
  [0, 14136, 1, 6],
  [0, 5132, 4, 5],


In [66]:
#去掉record数量小于5的session 去掉session数量少于3的用户 得到S_u2
S_u1={uid:[] for uid in uid_categories}
for uid in S_u.keys():
    for session in S_u[uid]:
        if(len(session)>=5):
            S_u1[uid].append(session)

In [67]:
S_u2={uid:S_u1[uid] for uid in uid_categories if len(S_u1[uid])>=5}
len(S_u2)

581

In [68]:
data_all['sessions']=S_u2

In [50]:
# coding: utf-8
from __future__ import print_function
from __future__ import division

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [70]:
parameters={}
parameters['uid_size']=len(uid_categories)
parameters['day_size']=7
parameters['daytime_size']=8
parameters['vid_size']=len(vid_categories)
parameters['vcid_size']=len(vcid_categories)

In [71]:
class Deepmove(nn.Module):
    def __init__(self,parameters):
        super().__init__()
        # self.uid_embsize=50
        # self.day_embsize=5
        # self.daytime_embsize=5

        self.uid_embed=nn.Embedding(parameters['uid_size'],50)
        self.day_embed=nn.Embedding(parameters['day_size'],5)
        self.daytime_embed=nn.Embedding(parameters['daytime_size'],5)
        self.vid_embed=nn.Embedding(parameters['vid_size'],500)
        self.vcid_embed=nn.Embedding(parameters['vcid_size'],10)

        self.fc_vid=nn.Linear(500,parameters['vid_size'])
        self.fc_vcid=nn.Linear(500,parameters['vcid_size'])
        self.fc=nn.Linear(550,500)

        self.rnn=nn.GRU(570,500,1)
        self.dropout = nn.Dropout(p=0.2)
    def forward(self,uid,trace):
        h1 = torch.zeros(1, 1, 500)

        uid_embed=self.uid_embed(traces['uid'])
        day_embed=self.day_embed(traces['day'])
        daytime_embed=self.daytime_embed(traces['daytime'])
        vid_embed=self.vid_embed(traces['vid'])
        vcid_embed=self.vcid_embed(traces['vcid'])

        x=torch.cat((day_embed,daytime_embed,vid_embed,vcid_embed),2)
        out, h1 = self.rnn(x, h1)
        out=torch.cat((uid_embed,out),2)
        out = out.squeeze()
        out=self.fc(out)
        
        out = self.dropout(out)
        y = self.fc_vcid(out)
        score = F.log_softmax(y,dim=None)

In [51]:
x=torch.Tensor([[1,2],[3,4]])
x[:,1]

tensor([2., 4.])

In [72]:
traces={uid:[] for uid in S_u2.keys()}
for uid in S_u2.keys():
    for i,session in enumerate(S_u2[uid]):
        if i==0:
            continue
        current_vid=torch.Tensor(session)[:,1].unsqueeze(-1)
        current_day=torch.Tensor(session)[:,2].unsqueeze(-1)
        current_daytime=torch.Tensor(session)[:,3].unsqueeze(-1)

        history_vid=[]
        history_day=[]
        history_daytime=[]
        for j in range(i):
            tmp_session=S_u2[uid][j]
            history_vid.extend([x[1] for x in tmp_session])
            history_day.extend([x[2] for x in tmp_session])
            history_daytime.extend([x[3] for x in tmp_session])
        history_vid=torch.Tensor(history_vid).unsqueeze(-1)
        history_day=torch.Tensor(history_day).unsqueeze(-1)
        history_daytime=torch.Tensor(history_daytime).unsqueeze(-1)

        trace={'current_vid':current_vid,'current_day':current_day,'current_daytime':current_daytime,'history_vid':history_vid,'history_day':history_day,'history_daytime':history_daytime}
        traces[uid].append(trace)

In [80]:
data_all['traces']=traces

In [81]:
uid_embed=nn.Embedding(parameters['uid_size'],parameters['vid_size'])

day_embed=nn.Embedding(parameters['day_size'],5)
daytime_embed=nn.Embedding(parameters['daytime_size'],5)
vid_embed=nn.Embedding(parameters['vid_size'],500)
# vcid_embed=nn.Embedding(parameters['vcid_size'],10)

fc=nn.Linear(510,500)
fc_vid=nn.Linear(500,parameters['vid_size'])
# fc_vcid=nn.Linear(500,parameters['vcid_size'])

rnn=nn.GRU(570,500,1)
dropout = nn.Dropout(p=0.2)

In [82]:
for uid,sessions in traces.items():
    for trace in sessions:
        h1 = torch.zeros(1, 1, 500)
        
        uid_embed=self.uid_embed(torch.Tensor([uid]).unsqueeze(0))
        day_embed=self.day_embed(trace['current_day'])
        daytime_embed=self.daytime_embed(trace['current_daytime'])
        vid_embed=self.vid_embed(trace['current_vid'])
        # vcid_embed=self.vcid_embed(trace['vcid'])

        x=torch.cat((day_embed,daytime_embed,vid_embed),2)
        out, h1 = self.rnn(x, h1)
        out=out[-1].unsqueeze(0)
        # out=torch.cat((uid_embed,out),2)
        # out = out.squeeze(1)
        out=self.fc(out)

        out = self.dropout(out)
        y = self.fc_vid(out)+uid_embed
        score = F.log_softmax(y,dim=None)

TypeError: cannot unpack non-iterable numpy.int64 object